In [1]:
import pandas as pd
import requests
import itertools
from bs4 import BeautifulSoup
from requests.exceptions import ChunkedEncodingError

# Trial Proxies

In [2]:
# import requests
# from bs4 import BeautifulSoup
# import pandas as pd
# import itertools
# from requests.exceptions import ChunkedEncodingError

# # Read proxies from the file and create a cycle iterator
# with open('proxies.txt', 'r') as f:
#     proxies = [line.strip() for line in f if line.strip()]

# # Check if proxies are being read correctly
# if not proxies:
#     print("No valid proxies found in proxies.txt")
# else:
#     print(f"Proxies loaded: {proxies}")

# proxy_cycle = itertools.cycle(proxies)

# # Initialize an empty DataFrame to store results
# df = pd.DataFrame()

# # Loop over the desired number of pages
# for page_num in range(1, 5):
#     proxy = next(proxy_cycle)
#     print(f"Using proxy: {proxy} for page {page_num}")

#     # Set up request with proxy
#     headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
#     url = f'https://www.halfbakedharvest.com/category/recipes/type-of-meal/main-course/page/{page_num}/'
#     proxy_dict = {'http': proxy, 'https': proxy}

#     try:
#         response = requests.get(url, headers=headers, proxies=proxy_dict, timeout=30)

#         # Check if the request was successful
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.content, 'html.parser')
#             print(f"Successfully retrieved page {page_num} with proxy {proxy}")
#         else:
#             print(f"Failed to retrieve page {page_num} with proxy {proxy}. Status code: {response.status_code}")
#     except (requests.ConnectionError, requests.Timeout) as e:
#         print(f"Proxy {proxy} failed for page {page_num}: {e}")
#     except ChunkedEncodingError as e:
#         print(f"ChunkedEncodingError occurred for proxy {proxy} and page {page_num}: {e}")
#         continue




In [3]:
# Empty Dataframe
columns = [
    'title',
    'prep_time',
    'cook_time',
    'total_time',
    'ingredients',
    'ingredients with measurement',
    'instructions',
    'calories',
    'servings',
    'images'
]


In [4]:
final = pd.DataFrame(columns=columns)
final

,title,prep_time,cook_time,total_time,ingredients,ingredients with measurement,instructions,calories,servings,images


In [5]:
failed_url = []

# Extract Function

In [12]:
def extract(soup):
    cards = soup.find_all(class_='post-summary primary')
    recipe_pg = pd.DataFrame(columns=columns)
    proxy_cycle2 = itertools.cycle(proxies)
    
    title = []
    prep_time = []
    cook_time = []
    total_time = []
    
    ingredients_measurment = []
    ingredients = []
    instructions = []
    
    calories = []
    servings = []
    images = []

    for card in cards:
        proxy2 = next(proxy_cycle2)
        proxy_dict2  = {'http': proxy2, 'https': proxy2}
        link_tag = card.find('a')
        recipe_url = link_tag['href']
        try:
            response = requests.get(recipe_url, headers=headers, proxies=proxy_dict2, timeout=30)
            response.raise_for_status()  
            temp_soup = BeautifulSoup(response.text, 'lxml')

            # TITLE
            h2_tag = temp_soup.find('h2', class_='wprm-recipe-name')
            if h2_tag:
                title.append(h2_tag.text.strip())
            else:
                title.append('-')
            

            # PREP TIME
            prep_time_elem = temp_soup.find(class_='wprm-recipe-prep-time-container')
            if prep_time_elem:
                time_dict = {}
                hours_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-hours')
                minutes_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-minutes')

                if hours_tag:
                    hours_text = hours_tag.text.strip() if hours_tag else None
                    hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                    time_dict['hours'] = hours_value
                if minutes_tag:
                    minutes_text = minutes_tag.text.strip() if minutes_tag else None
                    minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                    time_dict['minutes'] = minutes_value
                prep_time.append(time_dict) 
            else:
                prep_time.append('-')


            # COOK TIME
            cook_time_elem = temp_soup.find(class_='wprm-recipe-cook-time-container')
            if cook_time_elem:
                time_dict = {}
                hours_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-hours')
                minutes_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-minutes')

                if hours_tag:
                    hours_text = hours_tag.text.strip() if hours_tag else None
                    hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                    time_dict['hours'] = hours_value
                if minutes_tag:
                    minutes_text = minutes_tag.text.strip() if minutes_tag else None
                    minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                    time_dict['minutes'] = minutes_value

                cook_time.append(time_dict) 
            else:
                cook_time.append('-')


            # TOTAL TIME
            total_time_elem = temp_soup.find(class_='wprm-recipe-total-time-container')
            if total_time_elem:
                time_dict = {}
                hours_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-hours')
                minutes_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-minutes')

                if hours_tag:
                    hours_text = hours_tag.text.strip() if hours_tag else None
                    hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                    time_dict['hours'] = hours_value
                if minutes_tag:
                    minutes_text = minutes_tag.text.strip() if minutes_tag else None
                    minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                    time_dict['minutes'] = minutes_value
                total_time.append(time_dict) 
            else:
                total_time.append('-')
            
            
            # INGREDIENTS
            ingr_list = temp_soup.find_all('li', class_='wprm-recipe-ingredient')
            if ingr_list:
                ingredient_dict ={}
                ingredients_recipe = []
                for ingr in ingr_list:
                    ingr_amount_elem = ingr.find(class_='wprm-recipe-ingredient-amount')
                    ingr_amount = ingr_amount_elem.text.strip() if ingr_amount_elem else None
                    ingr_name_elem = ingr.find(class_='wprm-recipe-ingredient-name')
                    ingr_name = ingr_name_elem.text.strip() if ingr_name_elem else None
                    ingredient_dict[ingr_name] = ingr_amount if ingr_amount else "seasoning"
                    ingredients_recipe.append(ingr_name)
                ingredients_measurment.append(ingredient_dict)
                ingredients.append(ingredients_recipe)
            else:
                ingredients_measurment.append('-')
                ingredients.append('-')
            
            
            # INSTRUCTIONS
            recipe_instr = temp_soup.find(class_='wprm-recipe-instruction-group')
            if recipe_instr:
                recipe_steps = []
                steps = recipe_instr.find_all("span", attrs={"class": None})
                for step in steps:
                    recipe_steps.append(step.text.strip())
                instructions.append(recipe_steps)
            else:
                instructions.append('-')
                
            
            # SERVINGS
            serving_div = temp_soup.find('span', class_='wprm-recipe-servings')
            if serving_div:
                servings.append(serving_div.text.strip())
            else:
                servings.append('-')

            # CALORIES
            calorie_div = temp_soup.find('span',class_='wprm-recipe-nutrition-with-unit')
            if calorie_div:
                calories.append(calorie_div.text.strip())
            else:
                calories.append('-')
            
            # IMAGES
            images.append('-')
            img_div = temp_soup.find(class_='entry-content')
            img_tags = img_div.find_all('img')
            jpg_links = [img['src'] for img in img_tags if img['src'].endswith('.jpg')]
            if len(jpg_links) >= 2:
                second_jpg_link = jpg_links[1]
                images[-1] = second_jpg_link

            
        except requests.exceptions.RequestException as e:
            print(f"Failed to fetch the page: {e}")
            failed_url.append(recipe_url)  # Append failed URL to list
            continue  # Move to the next card if there's an error
    
    
    
    
    recipe_pg['title'] = title
    recipe_pg['prep_time'] = prep_time
    recipe_pg['cook_time'] = cook_time
    recipe_pg['total_time'] = total_time
    recipe_pg['ingredients'] = ingredients
    recipe_pg['ingredients with measurement'] = ingredients_measurment
    recipe_pg['instructions'] = instructions
    recipe_pg['calories'] = calories
    recipe_pg['servings'] = servings
    recipe_pg['images']= images
    
    
    
    

In [13]:
# Read proxies from the file and create a cycle iterator
with open('proxies.txt', 'r') as f:
    proxies = [line.strip() for line in f if line.strip()]

# Check if proxies are being read correctly
if not proxies:
    print("No valid proxies found in proxies.txt")
else:
    print(f"Proxies loaded: {proxies}")

proxy_cycle1 = itertools.cycle(proxies)

# Initialize an empty DataFrame to store results
temp_recipes = pd.DataFrame(columns=columns)

# Loop over the desired number of pages
for page_num in range(1, 6):
    proxy1 = next(proxy_cycle1)

    # Set up request with proxy
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    url = f'https://www.halfbakedharvest.com/category/recipes/type-of-meal/main-course/page/{page_num}/'
    proxy_dict1 = {'http': proxy1, 'https': proxy1}

    try:
        response = requests.get(url, headers=headers, proxies=proxy_dict1, timeout=30)
        
        # Check if the request was successful
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            recipe_pg = extract(soup)
            temp_recipes = pd.concat([temp_recipes, recipe_pg], ignore_index=True)
            print(f"Successfully retrieved page {page_num} with proxy {proxy}")
        else:
            print(f"Failed to retrieve page {page_num} with proxy {proxy1}. Status code: {response.status_code}")
    except (requests.ConnectionError, requests.Timeout) as e:
        print(f"Proxy {proxy} failed for page {page_num}: {e}")
    except ChunkedEncodingError as e:  # Handle ChunkedEncodingError
        print(f"ChunkedEncodingError occurred for proxy {proxy1} and page {page_num}: {e}")
        continue  # Move to the next page



Proxies loaded: ['221.140.235.237:5002', '117.250.3.58:8080', '189.240.60.171:9090', '189.240.60.166:9090', '34.92.250.88:11111', '189.240.60.164:9090', '189.240.60.169:9090', '189.240.60.163:9090']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Successfully retrieved page 5 with proxy 221.140.235.237:5002


In [ ]:
proxy = '172.183.241.1:8080'
proxy_dict = {'http': proxy, 'https': proxy}


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15'}
url = f'https://www.halfbakedharvest.com/category/recipes/type-of-meal/main-course/page/2/'

webpage = requests.get(url, headers=headers, proxies=proxy_dict, timeout=5)
if webpage.status_code == 200:
    print(f"Successfully retrieved page 2 with proxy {proxy}")
else:
    print(f"Failed to retrieve page 2 with proxy {proxy}. Status code: {webpage.status_code}")

In [ ]:
webpage = webpage.text

In [ ]:
soup = BeautifulSoup(webpage, 'lxml')

In [ ]:
print(soup.prettify())

In [ ]:
cards = soup.find_all(class_='post-summary primary')
cards

In [ ]:
for card in cards:
    link_tag = card.find('a')
    print(link_tag['href'])

In [ ]:
title = []
for card in cards:
    link_tag = card.find('a')
    recipe_url = link_tag['href']
    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        temp_soup = BeautifulSoup(response.text, 'lxml')

        h2_tag = temp_soup.find('h2', class_='wprm-recipe-name')
        if h2_tag:
            title.append(h2_tag.text.strip())
        else:
            title.append('-')

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        continue  # Move to the next card if there's an error


print(len(title))
print(title)

In [ ]:
# hours_tag = soup.find('span', class_='wprm-recipe-details-hours')
# minutes_tag = soup.find('span', class_='wprm-recipe-details-minutes')

# hours = hours_tag.get_text(strip=True)
# minutes = minutes_tag.get_text(strip=True)

# hours_value = int(hours.split()[0])
# minutes_value = int(minutes.split()[0])

# time_dict = {'hours': hours_value, 'minutes': minutes_value}
# print(time_dict)

In [ ]:
prep_time = []
cook_time = []
total_time = []

for card in cards:
    link_tag = card.find('a')
    recipe_url = link_tag['href']
    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        temp_soup = BeautifulSoup(response.text, 'lxml')
        
        prep_time_elem = temp_soup.find(class_='wprm-recipe-prep-time-container')
        cook_time_elem = temp_soup.find(class_='wprm-recipe-cook-time-container')
        total_time_elem = temp_soup.find(class_='wprm-recipe-total-time-container')

        # PREP TIME
        if prep_time_elem:
            time_dict = {}
            hours_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-hours')
            minutes_tag = prep_time_elem.find('span', class_='wprm-recipe-prep_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value
            prep_time.append(time_dict) 
        else:
            prep_time.append('-')
        
        
        # COOK TIME
        if cook_time_elem:
            time_dict = {}
            hours_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-hours')
            minutes_tag = cook_time_elem.find('span', class_='wprm-recipe-cook_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value

            cook_time.append(time_dict) 
        else:
            cook_time.append('-')
        
        
        # TOTAL TIME
        if total_time_elem:
            time_dict = {}
            hours_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-hours')
            minutes_tag = total_time_elem.find('span', class_='wprm-recipe-total_time-minutes')

            if hours_tag:
                hours_text = hours_tag.text.strip() if hours_tag else None
                hours_value = re.sub(r'\D', '', hours_text) if hours_text else None
                time_dict['hours'] = hours_value
            if minutes_tag:
                minutes_text = minutes_tag.text.strip() if minutes_tag else None
                minutes_value = re.sub(r'\D', '', minutes_text) if minutes_text else None
                time_dict['minutes'] = minutes_value
            total_time.append(time_dict) 
        else:
            total_time.append('-')
        
        

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        continue  # Move to the next card if there's an error


print(len(prep_time))
print(prep_time)

print(len(cook_time))
print(cook_time)

print(len(total_time))
print(total_time)

In [ ]:
df = pd.DataFrame()

In [ ]:
df['title'] = title
df['prep_time'] = prep_time
df['cook_time'] = cook_time
df['total_time'] = total_time

In [ ]:
df

In [ ]:
ingredients_measurment = []
ingredients = []

for card in cards:
    link_tag = card.find('a')
    recipe_url = link_tag['href']
    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        temp_soup = BeautifulSoup(response.text, 'lxml')
        
        ingr_list = temp_soup.find_all('li', class_='wprm-recipe-ingredient')
        
        ingredient_dict ={}
        ingredients_recipe = []
        for ingr in ingr_list:
            ingr_amount_elem = ingr.find(class_='wprm-recipe-ingredient-amount')
            ingr_amount = ingr_amount_elem.text.strip() if ingr_amount_elem else None
            ingr_name_elem = ingr.find(class_='wprm-recipe-ingredient-name')
            ingr_name = ingr_name_elem.text.strip() if ingr_name_elem else None
            ingredient_dict[ingr_name] = ingr_amount if ingr_amount else "seasoning"
            ingredients_recipe.append(ingr_name)
        
        ingredients_measurment.append(ingredient_dict)
        ingredients.append(ingredients_recipe)
        
        
        
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        continue  # Move to the next card if there's an error


print(len(ingredients_measurment))
print(ingredients_measurment)


print(len(ingredients))
print(ingredients)


In [ ]:
df['ingredients'] = ingredients
df['ingredients with measurement'] = ingredients_measurment
df

In [ ]:
instructions = []

for card in cards:
    link_tag = card.find('a')
    recipe_url = link_tag['href']
    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        temp_soup = BeautifulSoup(response.text, 'lxml')
        
        recipe_instr = temp_soup.find(class_='wprm-recipe-instruction-group')
        if recipe_instr:
            recipe_steps = []
            steps = recipe_instr.find_all("span", attrs={"class": None})
            for step in no_class_attribute:
                recipe_steps.append(step.text.strip())
            instructions.append(recipe_steps)
        else:
            instructions.append('-')
        
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        continue  # Move to the next card if there's an error

print(len(instructions))
print(instructions)

In [ ]:
df['instructions'] = instructions
df

In [ ]:
df.iloc[1]['instructions']

In [ ]:
calories = []
servings = []
for card in cards:
    link_tag = card.find('a')
    recipe_url = link_tag['href']
    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        temp_soup = BeautifulSoup(response.text, 'lxml')
        
        # Servings
        serving_div = temp_soup.find('span', class_='wprm-recipe-servings')
        if serving_div:
            servings.append(serving_div.text.strip())
        else:
            servings.append('-')
        
        # Calories
        calorie_div = temp_soup.find('span',class_='wprm-recipe-nutrition-with-unit')
        if calorie_div:
            calories.append(calorie_div.text.strip())
        else:
            calories.append('-')
        
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        continue  # Move to the next card if there's an error

print(servings)
print(calories)

In [ ]:
df['calories'] = calories
df['servings'] = servings
df

In [ ]:
# imgs = []

# for card in cards:
#     link_tag = card.find('a')
#     recipe_url = link_tag['href']
#     try:
#         response = requests.get(recipe_url, headers=headers)
#         response.raise_for_status()  # Raise an exception for non-200 status codes
#         temp_soup = BeautifulSoup(response.text, 'lxml')
        
#         # Image
#         img_div = temp_soup.find(class_='entry-content')
#         if img_div:
#             img_tag = img_div.find('img')
#             if img_tag:
#                 img_src = img_tag['src']
#                 print(img_src)
#             else:
#                 print("Image source not found.")
#         else:
#             print("Div not found")
        
#     except requests.exceptions.RequestException as e:
#         print(f"Failed to fetch the page: {e}")
#         continue  # Move to the next card if there's an error



In [ ]:
images = []

for card in cards:
    link_tag = card.find('a')
    recipe_url = link_tag['href']
    try:
        response = requests.get(recipe_url, headers=headers)
        response.raise_for_status()  # Raise an exception for non-200 status codes
        temp_soup = BeautifulSoup(response.text, 'lxml')
        
        # Image
        images.append('-')
        img_div = temp_soup.find(class_='entry-content')
        img_tags = img_div.find_all('img')

        # Extract the second link ending with ".jpg" if it exists
        jpg_links = [img['src'] for img in img_tags if img['src'].endswith('.jpg')]
        if len(jpg_links) >= 2:
            second_jpg_link = jpg_links[1]
            images[-1] = second_jpg_link
        

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch the page: {e}")
        continue  # Move to the next card if there's an error

print(len())
print(images)